In [44]:
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('data/audios/test.csv')

In [3]:
count = 100

not_same = []
for og, rec in zip(df['og_transcription'], df['ms_asr_transcription']):
    if count == 0:
        break
    
    if og != rec:
        not_same.append([og, rec])

        count -= 1


In [4]:
df2 = pd.DataFrame(not_same, columns=df.columns[1:])

In [5]:
df2.head()

,og_transcription,ms_asr_transcription
0,I have a sharp pain in my lower stomach.,I have a shopping in my lowest.
1,Don't cry,Don't try.
2,When I get out of bed in the morning my body f...,"When I get out of bed in the morning, my body ..."
3,i have a great pain in my thorax from heart in...,I have a great pain in my thorax from heart in...
4,I have a hard muscle pain since i went to the gym,I have a hard muscle pain since I went to the ...


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [7]:
MODEL_CHECKPOINT = "google/flan-t5-small-finetuned-asr_correction"

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

In [8]:
def correct(text):
    input_ids = tokenizer(text, return_tensors="pt")["input_ids"]
    output = model.generate(input_ids)
    translated_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    return translated_text

In [42]:
import re

def remove_characters(line):
    return re.sub('[!@#$.,]', '', line)

def find_diff_words(str1, str2):
    str1 =remove_characters(str1)
    str2 =remove_characters(str2)

    words1 = str1.split()
    words2 = str2.split()
    diff_words = []
    for i in range(min((len(words1), len(words2)))):
        if words1[i].lower() != words2[i].lower():
            # print(words1[i], words2[i])
            diffs = [i, words1[i], words2[i]]
            diff_words.append(diffs)
    return diff_words

In [ ]:
corrected = []
diff = []

for sent in tqdm(df2['ms_asr_transcription']):
    correct_sent = correct(sent)
    corrected.append(correct_sent)
    diff.append(find_diff_words(sent, correct_sent))


In [45]:
df2['corrected'] = corrected
df2['differences'] = diff

In [46]:
df2.to_csv('tested2.csv', index=False)